## Run a script which pulls about 7364 unique english language captions and their tfidf vectors from the MySQL databank and brings into scope the variables we need, a sparse matrix and also a matching dataframe with the names of the videos.

In [1]:
%%time
%run prepare_variables.py

CPU times: user 9.04 s, sys: 364 ms, total: 9.41 s
Wall time: 9.56 s


In [2]:
p.shape

(7364, 429429)

In [3]:
p

<7364x429429 sparse matrix of type '<class 'numpy.float64'>'
	with 1581374 stored elements in COOrdinate format>

#### That is very sparse - only 0.05% of the matrix has anything in it.

#### http://scikit-learn.org/stable/auto_examples/text/document_clustering.html#sphx-glr-auto-examples-text-document-clustering-py

#### http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html#sklearn.preprocessing.Normalizer

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
normalizer = Normalizer(copy=False)
from sklearn.cluster import KMeans
from numpy.random import random

In [22]:
%%time
svd = TruncatedSVD(n_components=1000, n_iter=7, random_state=42)
pipeline = make_pipeline(svd, normalizer)
p_new1 = pipeline.fit_transform(p)
p_new1.shape

CPU times: user 10min 12s, sys: 1min 37s, total: 11min 50s
Wall time: 5min 20s


In [6]:
%%time
svd = TruncatedSVD(n_components=2000, n_iter=7, random_state=42)
pipeline = make_pipeline(svd, normalizer)
p_new2 = pipeline.fit_transform(p)

CPU times: user 30min 35s, sys: 2min 15s, total: 32min 50s
Wall time: 12min 33s


In [24]:
%%time
svd = TruncatedSVD(n_components=150, n_iter=100, random_state=42)
pipeline = make_pipeline(svd, normalizer)
p_new3 = pipeline.fit_transform(p)

CPU times: user 8min 8s, sys: 2min 58s, total: 11min 7s
Wall time: 6min 31s


In [33]:
%%time
svd = TruncatedSVD(n_components=150, n_iter=100, random_state=42)
p_new4 = normalizer.fit_transform(p)

CPU times: user 72 ms, sys: 0 ns, total: 72 ms
Wall time: 71.1 ms


In [5]:
#  p_new = normalizer.fit_transform(p)

In [35]:
print (p_new1.shape, p_new2.shape, p_new3.shape, p_new4.shape)

(7364, 1000) (7364, 2000) (7364, 150) (7364, 429429)


In [ ]:
%%time
kmeans1 = KMeans(n_clusters=5).fit(p_new1) 

In [8]:
%%time
kmeans = KMeans(n_clusters=5).fit(p_new2) 

In [14]:
%%time
kmeans = KMeans(n_clusters=5).fit(p_new3) 

In [ ]:
%%time
kmeans = KMeans(n_clusters=5).fit(p_new4)

## How many videos has it found in each cluster?

In [ ]:
kmeans = kmeans3

In [31]:
videos_df['cluster_labels'] = kmeans.labels_
videos_df.groupby('cluster_labels').count()[['videoTitle']]

,videoTitle
cluster_labels,
0,1839
1,2037
2,2483
3,1005


## Sample ten videos randomly from each of those groups:

In [32]:
pd.set_option('display.max_rows', 120)

# https://stackoverflow.com/questions/22472213/python-random-selection-per-group

size = 10        # sample size
replace = False  # with replacement
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
clustered_sample = videos_df.groupby('cluster_labels', as_index=False).apply(fn)
clustered_sample[['videoTitle','cluster_labels', 'link']]

videoTitle  cluster_labels  \
0 6764            Humphreys Summer Cooking Tips with KUSI               0   
  1799  Mozart Minute: Chihiro Fukuda (Cooking and Cla...               0   
  783   DIY M&M's Banana Create M&M's Fruit Surprise E...               0   
  7324              Get Marty: Owner Of Service Dog Cited               0   
  5082         Nonlinear Data - Intro to Machine Learning               0   
  1045  Teaching in an Online Graduate Program  – Anno...               0   
  2367               Cooking Safety with Columbia Housing               0   
  1840             What's Cooking  Ezras Enlightened Cafe               0   
  3289  Funny Kids Game TV - Cooking Mama Let's Cook -...               0   
  917   Village Food factory -  Amazing village style ...               0   
1 3866       Food Review: Cooking chinese hot pot at home               1   
  3542  Regression Intro - Practical Machine Learning ...               1   
  1136           YumPower cooking demos at MOA for Go Red               1   
  6748                    CS:GO's Future from Valve's AMA               1   
  6006  Curry Chicken Sub | Thank You Speech | 咖哩雞潛艇 :...               1   
  2229  ENG SUBS Battalion Vostok fighter cooking bors...               1   
  868                     Cooking with Robonyan Episode 3               1   
  4980                   Cooking Time! Breakfast in a Cup               1   
  1233                    RebelWellness Is...Vanessa Cook               1   
  3140            MIMIC - Georgia Tech - Machine Learning               1   
2 7099  Head of Big Data at Nortal, Lauri Ilison speak...               2   
  3297                 Learn the Deep English Method Fast               2   
  6134                    The Capital Asset Pricing Model               2   
  4901  Insights | Big data & cognitive technologies t...               2   
  6684         4. Stochastic Gradient Descent Convergence               2   
  2533                          What is Machine Learning?               2   
  2675                                 2. Sliding Windows               2   
  4072               011 Machine learning for video games               2   
  521                                       Deep learning               2   
  5352  Machine Learning With Python - Hierarchical Cl...               2   
3 1662  Mix Vegetable Pakora Recipe in Hindi By Cookin...               3   
  2607    HOW TO COOKING JAPANESE MADE “PEYOUNG YAKISOBA”               3   
  1206  How I cooked Korean Mutton in Indian Style|Tam...               3   
  4962  Sichuan Chicken Recipe - Szechuan Chicken by C...               3   
  5408  Tsubako cooking vegetarian pyttipanna　ツバコクッキング...               3   
  47                          [ML1-006] Setting up Jupyer               3   
  652   Chocolate Marble Cake Recipe in Hindi by Cooki...               3   
  6464  Chicken Manchurian (how to make chicken manchu...               3   
  2803  Methi Pak / Methi ke Laddoo Recipe in Hindi By...               3   
  3011  How to Make Ginger Fried Chicken |  Hilah Cooking               3   

                                               link  
0 6764  https://www.youtube.com/watch?v=XaYhbEVoHWc  
  1799  https://www.youtube.com/watch?v=bYRK7YKntk8  
  783   https://www.youtube.com/watch?v=4ZSVMRbrrek  
  7324  https://www.youtube.com/watch?v=zTalPwbmXBM  
  5082  https://www.youtube.com/watch?v=PxE2bbG2Hkw  
  1045  https://www.youtube.com/watch?v=76YKkk8dWt4  
  2367  https://www.youtube.com/watch?v=Ee-erS8KIuQ  
  1840  https://www.youtube.com/watch?v=C7ApHn2hqZc  
  3289  https://www.youtube.com/watch?v=ibYV3C8OXQk  
  917   https://www.youtube.com/watch?v=66bFU50Wwb8  
1 3866  https://www.youtube.com/watch?v=KoiWzEPBCqU  
  3542  https://www.youtube.com/watch?v=JcI5Vnw0b2c  
  1136  https://www.youtube.com/watch?v=7Z6_cPKVWUU  
  6748  https://www.youtube.com/watch?v=x7D76z4k0Kk  
  6006  https://www.youtube.com/watch?v=Twg-Zp7iXOI  
  2229  https://www.youtu